<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 842ms/step - loss: 0.6754 - accuracy: 0.5864 - val_loss: 0.6564 - val_accuracy: 0.7561
Epoch 2/10
26/26 [==============================] - 21s 800ms/step - loss: 0.6286 - accuracy: 0.6813 - val_loss: 0.7342 - val_accuracy: 0.4780
Epoch 3/10
26/26 [==============================] - 21s 824ms/step - loss: 0.5960 - accuracy: 0.6825 - val_loss: 0.6331 - val_accuracy: 0.5805
Epoch 4/10
26/26 [==============================] - 21s 815ms/step - loss: 0.5198 - accuracy: 0.7555 - val_loss: 0.4861 - val_accuracy: 0.7366
Epoch 5/10
26/26 [==============================] - 21s 800ms/step - loss: 0.4831 - accuracy: 0.7676 - val_loss: 0.4703 - val_accuracy: 0.8195
Epoch 6/10
26/26 [==============================] - 21s 819ms/step - loss: 0.4253 - accuracy: 0.8090 - val_loss: 0.3842 - val_accuracy: 0.8244
Epoch 7/10
26/26 [==============================] - 23s 900ms/step - loss: 0.3731 - accuracy: 0.8370 - val_loss: 0.3319 - val_accuracy: 0.8634

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.64.h5
26/26 - 31s - loss: 0.6678 - accuracy: 0.6314 - val_loss: 0.6424 - val_accuracy: 0.6634
Epoch 2/5

Epoch 00002: saving model to weights.02-0.60.h5
26/26 - 24s - loss: 0.6349 - accuracy: 0.6302 - val_loss: 0.5985 - val_accuracy: 0.7073
Epoch 3/5

Epoch 00003: saving model to weights.03-0.57.h5
26/26 - 25s - loss: 0.5872 - accuracy: 0.7056 - val_loss: 0.5707 - val_accuracy: 0.6878
Epoch 4/5

Epoch 00004: saving model to weights.04-0.47.h5
26/26 - 24s - loss: 0.5481 - accuracy: 0.7324 - val_loss: 0.4712 - val_accuracy: 0.9024
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 24s - loss: 0.4643 - accuracy: 0.8187 - val_loss: 0.4114 - val_accuracy: 0.8244


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 29s - loss: 0.6744 - accuracy: 0.5414 - val_loss: 0.7102 - val_accuracy: 0.4488


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 28s - loss: 0.6774 - accuracy: 0.5815 - val_loss: 0.6831 - val_accuracy: 0.4390
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 22s - loss: 0.6527 - accuracy: 0.6204 - val_loss: 0.6389 - val_accuracy: 0.8390


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6691 - accuracy: 0.5718 - val_loss: 0.6565 - val_accuracy: 0.6927
Epoch 2/50
26/26 - 21s - loss: 0.6256 - accuracy: 0.7056 - val_loss: 0.6992 - val_accuracy: 0.4585
Epoch 3/50
26/26 - 21s - loss: 0.5844 - accuracy: 0.7251 - val_loss: 0.5743 - val_accuracy: 0.7366
Epoch 4/50
26/26 - 21s - loss: 0.5385 - accuracy: 0.7676 - val_loss: 0.5472 - val_accuracy: 0.7268
Epoch 5/50
26/26 - 21s - loss: 0.4686 - accuracy: 0.8273 - val_loss: 0.4378 - val_accuracy: 0.8683
Epoch 6/50
26/26 - 20s - loss: 0.4023 - accuracy: 0.8443 - val_loss: 0.3437 - val_accuracy: 0.8537
Epoch 7/50
26/26 - 21s - loss: 0.3637 - accuracy: 0.8650 - val_loss: 0.4418 - val_accuracy: 0.8244
Epoch 8/50
26/26 - 21s - loss: 0.3018 - accuracy: 0.8881 - val_loss: 0.2400 - val_accuracy: 0.9317
Epoch 9/50
26/26 - 21s - loss: 0.2696 - accuracy: 0.8990 - val_loss: 0.2414 - val_accuracy: 0.8732
Epoch 10/50
26/26 - 21s - loss: 0.1940 - accuracy: 0.9440 - val_loss: 0.1690 - val_accuracy: 0.9512
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 787ms/step - loss: 0.6679 - accuracy: 0.5949 - val_loss: 0.6304 - val_accuracy: 0.7659
Epoch 2/5
26/26 [==============================] - 21s 804ms/step - loss: 0.5986 - accuracy: 0.7202 - val_loss: 0.5826 - val_accuracy: 0.7561
Epoch 3/5
26/26 [==============================] - 20s 776ms/step - loss: 0.5680 - accuracy: 0.7433 - val_loss: 0.6330 - val_accuracy: 0.5805
Epoch 4/5
26/26 [==============================] - 20s 770ms/step - loss: 0.5253 - accuracy: 0.7482 - val_loss: 0.4741 - val_accuracy: 0.8439
Epoch 5/5
26/26 [==============================] - 21s 791ms/step - loss: 0.4548 - accuracy: 0.8054 - val_loss: 0.5334 - val_accuracy: 0.6927


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.594891,0.668350,0.765854,0.630392
1,1,0.720195,0.599174,0.756098,0.582553
2,2,0.743309,0.567936,0.580488,0.633031
3,3,0.748175,0.526428,0.843902,0.474112
4,4,0.805353,0.451195,0.692683,0.533446


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 1
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 33s 1s/step - loss: 0.6713 - accuracy: 0.5766 - val_loss: 0.6859 - val_accuracy: 0.4488

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 772ms/step - loss: 0.6412 - accuracy: 0.6752 - val_loss: 0.6644 - val_accuracy: 0.5317

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6244 - accuracy: 0.7190 - val_loss: 0.6442 - val_accuracy: 0.5902

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 762ms/step - loss: 0.6156 - accuracy: 0.7433 - val_loss: 0.6333 - val_accuracy: 0.6780

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 781ms/step - loss: 0.6108

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 841ms/step - loss: 0.6734 - accuracy: 0.5730 - val_loss: 0.6466 - val_accuracy: 0.7707
Epoch 2/50
26/26 [==============================] - 21s 800ms/step - loss: 0.6284 - accuracy: 0.6569 - val_loss: 0.6099 - val_accuracy: 0.6829
Epoch 3/50
26/26 [==============================] - 20s 777ms/step - loss: 0.5539 - accuracy: 0.7360 - val_loss: 0.5332 - val_accuracy: 0.8146
Epoch 4/50
26/26 [==============================] - 21s 801ms/step - loss: 0.4785 - accuracy: 0.8078 - val_loss: 0.4629 - val_accuracy: 0.8439
Epoch 5/50
26/26 [==============================] - 20s 773ms/step - loss: 0.4387 - accuracy: 0.8187 - val_loss: 0.4087 - val_accuracy: 0.8488
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.3688 - accuracy: 0.8662
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 774ms/step - loss: 0.3700 - accuracy: 0.8650 - val_loss: 0.6388 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4719), started 0:16:47 ago. (Use '!kill 4719' to kill it.)